In [ ]:
import pandas as pd
import re
from rapidfuzz import process

This notebook has been used to preprocess the data. Since the data has been stored differently from each website this is quite a mess that needed to be uniquely changed based on the dataset. 

In [ ]:
Torzon = pd.read_csv('data/hash.csv')
# Extract the last three characters from the price column as currency
# Drop the first column
df_Dark = Torzon.drop(columns=Torzon.columns[0])

# Rename the columns
df_Dark.columns = ["name", "price", "seller", "origin", "destination"]

# Extract the last three characters from the price column as currency
df_Dark["price"] = df_Dark["price"].str[:-1]
df_Dark["destination"] = df_Dark["destination"].str[3:]
df_Dark["origin"] = df_Dark["origin"].str[3:]
#df_Dark['price'] = pd.to_numeric(df_Dark['price'], errors='coerce')

# Clean the 'price' column
df_Dark['price'] = df_Dark['price'].str.replace(r'\s+', '', regex=True)  # Remove all spaces
df_Dark['price'] = df_Dark['price'].str.replace(r'From', '', regex=True)  # Remove the word "From"
df_Dark['price'] = df_Dark['price'].str.replace(r'USD', '', regex=True)  # Remove the currency "USD"

# Convert the cleaned price column to numeric
#df_Dark['price'] = pd.to_numeric(df_Dark['price'], errors='coerce')
# Extract everything until a ',' or '(' in each row of the column
df_Dark['destination'] = df_Dark['destination'].str.extract(r'^([^,(]+)')
df_Dark['origin'] = df_Dark['origin'].str.extract(r'^([^,(]+)')

# Display the updated DataFrame

df_Dark

,name,origin,destination,seller,price,amount_of_drugs
0,10g Hash Premium (3x filtré) | 120ug | Trop x ...,France,European Union,FourDeFrance,"130,00",10.0
1,10g Hash Premium (3x filtré) | Head Banger | 1...,France,European Union,FourDeFrance,"130,00",10.0
2,10g Hash Premium (3x filtré) | Strawberry Kush...,France,European Union,FourDeFrance,"130,00",10.0
3,10G Hash!!,France,France,WeedShopHulk,"85,00",10.0
4,14G Squid Game Haute Qualité,France,World Wide,Chapo,"100,00",14.0
...,...,...,...,...,...,...
60,[LEMON] 100G MAROCAIN HQ [HASHISH],France,France,RizLaCroix,"450,00",100.0
61,[LEMON] 10G MAROCAIN HQ [HASHISH],France,France,RizLaCroix,"70,00",10.0
62,[LEMON] 25G MAROCAIN HQ [HASHISH],France,France,RizLaCroix,"170,00",25.0
63,[LEMON] 50G MAROCAIN HQ [HASHISH],France,France,RizLaCroix,"260,00",50.0


In [116]:
# Step 2: Load and prepare the strains.csv dataset
df = pd.read_csv("data/strains.csv")

# Preprocess: keep only relevant columns
df = df[['Strain', 'Type']].dropna()
df['Strain'] = df['Strain'].str.lower().str.strip()
df['Type'] = df['Type'].str.lower().str.strip()
df

,Strain,Type
0,100-og,hybrid
1,98-white-widow,hybrid
2,1024,sativa
3,13-dawgs,hybrid
4,24k-gold,hybrid
...,...,...
2346,zeus-og,hybrid
2347,zkittlez,indica
2348,zombie-kush,indica
2349,zombie-og,indica


In [ ]:
#match the strain names in the darkweb dataset with the strain list using fuzzy matching

def fuzzy_find_strain(text, strain_list, threshold=50):

    # Clean the text: remove unwanted characters and normalize
    text = re.sub(r"[^\w\s]", " ", text.lower())  # Remove non-alphanumeric characters
    text = re.sub(r"\s+", " ", text)  # Normalize spaces
    text = text.strip()

    # Fuzzy match the input text against the strain list
    result = process.extractOne(text, strain_list, score_cutoff=threshold)
    
    # Return the match if found, else return 'unknown'
    if result:
        match, score, _ = result
        return match.title()  # Title case the matched strain name
    else:
        return "unknown"

def extract_info_with_strain_lookup(text, strain_list):
    text = text.lower()

    # 1. Extract weight range or single weight
    weight_matches = re.findall(r"(\d+(?:\.\d+)?)[\s\-–to]{1,4}(\d+(?:\.\d+)?)(?:\s*)?(g|gr|grams)?", text)
    single_weight_match = re.findall(r"(\d+(?:\.\d+)?)(?:\s*)?(g|gr|grams|\(oz\)|oz|\$)", text)

    low_weight = high_weight = None
    if weight_matches:
        w = weight_matches[0]
        low_weight = f"{w[0]}g"
        high_weight = f"{w[0]}g"
    elif single_weight_match:
        weights = [float(w[0]) for w in single_weight_match]
        max_weight = max(weights)
        high_weight = f"{max_weight}g"
        low_weight = ""

    # 2. Fuzzy strain matching
    strain = fuzzy_find_strain(text, strain_list)

    return (low_weight, high_weight, strain.title())

strain_list = df["Strain"].dropna().str.lower().str.strip().unique().tolist()
all_tuples = []
for e in df_Dark['name']:
    all_tuples.append(extract_info_with_strain_lookup(e,strain_list))


# Unpack the tuples into separate columns
df_Dark['Low_weight'], df_Dark['High_weight'], df_Dark['Strain'] = zip(*all_tuples)
df_Dark = df_Dark.dropna().reset_index(drop=True)
df_Dark

,name,origin,destination,seller,price,amount_of_drugs,Low_weight,High_weight,Strain
0,10g Hash Premium (3x filtré) | 120ug | Trop x ...,France,European Union,FourDeFrance,"130,00",10.0,,10.0g,Blue-Magic
1,10g Hash Premium (3x filtré) | Head Banger | 1...,France,European Union,FourDeFrance,"130,00",10.0,,10.0g,Headbanger
2,10g Hash Premium (3x filtré) | Strawberry Kush...,France,European Union,FourDeFrance,"130,00",10.0,,10.0g,Strawberry
3,10G Hash!!,France,France,WeedShopHulk,"85,00",10.0,,10.0g,Ash
4,14G Squid Game Haute Qualité,France,World Wide,Chapo,"100,00",14.0,,14.0g,1024
5,200g Drysift White Widow Hash - 150 µ,France,European Union,PapaGanja,"1 150,00",200.0,,200.0g,White-Widow
6,20G Hash!!,France,France,WeedShopHulk,"155,00",20.0,,20.0g,Ash
7,25g Hash Dry ~20% THC,France,France,LePetitPrince,"159,98",25.0,,25.0g,Ash
8,25g Hash HQ,France,World Wide,Copek,"200,00",25.0,,25.0g,Ash
9,25G Squid Game Haute Qualité,France,World Wide,Chapo,"160,00",25.0,,25.0g,Juliet


In [ ]:
# Function to split and reassign values
def split_price(row):
    if '-' in row['price']:
        low, high = row['price'].split('-')
        row['high_price'] = high
        row['price'] = low
    else:
        row['high_price'] = row['price']  # Or you can set it to the same value as 'price'
    return row

# Apply the function to each row
df_Dark_splitted = df_Dark.apply(split_price, axis=1)


TypeError: argument of type 'float' is not iterable

In [ ]:
df_Dark.drop_duplicates()

# Drop the columns 'Low_weight' and 'price'
df_Dark = df_Dark.drop(columns=['amount_of_drugs', 'Low_weight'])

# Rename the column 'High_weight' to 'weight'
df_Dark = df_Dark.rename(columns={'High_weight': 'weight'})

# Display the updated DataFrame
# Load the strains.csv dataset
strains_df = pd.read_csv('data/strains.csv')

# Ensure the column names match and are in the same format
strains_df['Strain'] = strains_df['Strain'].str.lower().str.strip()
df_Dark['Strain'] = df_Dark['Strain'].str.lower().str.strip()

# Merge the datasets on the 'Strain' column
merged_df = df_Dark.merge(strains_df, on='Strain', how='left')
#merged_df.to_csv('data/cocoricohash_final.csv', index=False)

In [ ]:
def parse_seller_info(seller_string):
    # Remove the first two characters
    seller_string = seller_string[1:]
    
    # Find the name (characters before the first '(')
    name_end = seller_string.find('(')
    name = seller_string[:name_end].strip()
    
    # Find items_sold (integer between the first '(' and ')')
    items_sold_start = name_end + 1
    items_sold_end = seller_string.find(')', items_sold_start)
    items_sold = int(seller_string[items_sold_start:items_sold_end].strip())
    
    # Find rating (float after the next '(')
    rating_start = seller_string.find('(', items_sold_end) + 1
    rating_end = seller_string.find(')', rating_start)
    rating_str = seller_string[rating_start:rating_end].strip()
    
    # Remove non-numeric characters from the rating
    Rating_seller = float(re.sub(r"[^\d.]", "", rating_str))
    
    return name, items_sold, Rating_seller

# Example usage
seller_string = "\tSantalblanc (50) (5 ★)"
name, items_sold, Rating_seller = parse_seller_info(seller_string)
print(f"Name: {name}, Items Sold: {items_sold}, Rating_seller: {Rating_seller}")
#merged_df.to_csv('data/placeholder.csv', index=False)

Name: Santalblanc, Items Sold: 50, Rating_seller: 5.0


In [83]:
# Apply the function to the 'seller' column
merged_df[['seller', 'items_sold', 'Rating_seller']] = merged_df['seller'].apply(
    lambda x: pd.Series(parse_seller_info(x))
)

In [ ]:
# Initialize a list to store new rows
new_rows = []

# Iterate over the DataFrame to create new rows
for index, row in df_Dark_splitted.iterrows():
    if pd.notnull(row['high_price']):
        # Create a new row for the high price
        new_row = row.copy()
        new_row['price'] = row['high_price']
        new_row['Low_weight'] = row['High_weight']
        new_rows.append(new_row)

        # Modify the original row to keep the low price
        df_Dark_splitted.at[index, 'High_weight'] = None

# Create a DataFrame from the new rows
new_df = pd.DataFrame(new_rows)

# Concatenate the original and new DataFrames
result_df = pd.concat([df_Dark_splitted, new_df], ignore_index=True)

# Drop the high_price column as it's no longer needed
result_df = result_df.drop(columns=['high_price', 'High_weight'])

# Display the resulting DataFrame
result_df = result_df.drop_duplicates()
# Check the number of unique names in the 'name' column
#unique_names_count = result_df["name"].nunique()

#print(f"Number of unique names: {unique_names_count}")

result_df = result_df.replace(r'^\s*$', float('NaN'), regex=True)

result_df = result_df.dropna().reset_index(drop=True)
result_df

Number of unique names: 106


,name,price,seller,origin,destination,currency,Low_weight,Strain
0,5-100gr x Fruit Joy Static Tech - Hash - 52% T...,40,TopWeedOnlyOfficial,Germany,Europe,EUR,5g,Grapefruit
1,2-50gr x Fresh Blonde Jardella Hash 2g,20,TopCat,United Kingdom,United Kingdom,GBP,2g,Delahaze
2,0.5-50gr x BISCOTTI DRYSIFT HASH - 58% THC,18,DieseGute_,Germany,Worldwide,EUR,0.5g,Ash
3,2-500gr x HASH HQ,20,Copek,France,Worldwide,EUR,2g,Ash
4,3-28gr x Old Skool OG Ice Hash,15,StrainPirate,United States,United States,USD,3g,Aceh
...,...,...,...,...,...,...,...,...
160,10-50gr x NEW HASH STATIC BANANA AMNESIA 120u ...,380,jackmesrine,France,Worldwide,EUR,50g,Amnesia
161,5-250gr x 💎 Hash | Dry Sift | DutchDeluxe,1450,DutchDeluxe,Netherlands,Worldwide,EUR,250g,Blue-Hash
162,5-250gr x 💎 Hash | La Mousse - Lemon Kush | Du...,850,DutchDeluxe,Netherlands,Worldwide,EUR,250g,Lemon-Kush
163,7-1000gr x UKPABLO SPANISH IMPORT HASH,2950,UKPABLO,United Kingdom,United Kingdom,GBP,1000g,Trash


In [ ]:
# Define the exchange rates to EUR
exchange_rates = {
    "USD": 0.91,  # Example: 1 USD = 0.91 EUR
    "GBP": 1.14,  # Example: 1 GBP = 1.14 EUR
    "EUR": 1.00,   # EUR remains the same
    "AUD": 0.60,  # Example: 1 AUD = 0.60 EUR
    "CHF": 1.02,  # Example: 1 CHF = 1.02 EUR
    "CAD": 0.63  # Example: 1 INR = 0.011 EUR
}

# Convert the price column to numeric (if not already)
result_df["price"] = pd.to_numeric(result_df["price"], errors="coerce")

# Multiply the price by the exchange rate based on the currency
result_df["price_in_eur"] = result_df.apply(
    lambda row: row["price"] * exchange_rates.get(row["currency"], 1), axis=1
)

# Display the updated DataFrame
result_df = result_df.drop(columns=["price", "currency"])
# Load the strains.csv dataset
strains_df = pd.read_csv('data/strains.csv')

# Ensure the column names match and are in the same format
strains_df['Strain'] = strains_df['Strain'].str.lower().str.strip()
result_df['Strain'] = result_df['Strain'].str.lower().str.strip()

# Merge the datasets on the 'Strain' column
merged_df = result_df.merge(strains_df, on='Strain', how='left')

# Display the merged DataFrame
#merged_df.to_csv('data/placeholder.csv', index=False)